In [16]:
import pandas as pd
import numpy as np
from numpy import argmax
from numpy import sqrt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
from sklearn.utils import resample
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from statistics import median
import pickle
import csv
import multiprocess

In [17]:
MAX_BATCH = [2, 4, 8, 16]
algorithm = ['BATCHSTOP4']

In [18]:
projects = ['rails.csv', 'jruby.csv', 'metasploit-framework.csv', 'cloudify.csv', 'vagrant.csv', 'rubinius.csv', 'open-build-service.csv', 'gradle.csv', 'sonarqube.csv', 'loomio.csv', 'fog.csv', 'opal.csv', 'cloud_controller_ng.csv', 'puppet.csv', 'concerto.csv', 'sufia.csv', 'geoserver.csv', 'orbeon-forms.csv', 'graylog2-server.csv', 'heroku.csv']
data_path = '../data/'
confidence = list(range(2,21,1))

In [19]:
result_file = open('19_9_batchstop4.csv', 'w')
result_headers = ['project', 'algorithm', 'batch_size', 'confidence', 'project_reqd_builds', 'project_missed_builds', 'project_build_duration', 'project_saved_builds', 'project_delays', 'testall_size', 'batch_delays']
writer = csv.writer(result_file)
writer.writerow(result_headers)

168

In [20]:
def get_train_test_data(filename):
    
    csv_file = csv.reader(open(filename, 'r'))
    temp_data = []
    final_data = []

    for item in csv_file:
        temp_data.append(item)

    for i in range(len(temp_data[0])):
        temp = []
        for index in range(1, len(temp_data)):
            temp.append(temp_data[index][i])
        final_data.append(temp)

    indices = range(len(final_data[3]))

    #capture the metrics of source churn, test churn, file churn and team size in a list
    src_churn = []
    file_churn = []
    test_churn = []
    team_size = []
    build_result = []
    git_num_all_built_commits = []
    gh_num_commits_on_files_touched = []
    argument = []

    for index in indices:
        src_churn.append(float(final_data[23][index]))
        file_churn.append(float(final_data[27][index]))
        test_churn.append(float(final_data[24][index]))
        team_size.append(float(final_data[14][index]))
        
        if final_data[42][index] == 'passed':
            build_result.append(1)
        else:
            build_result.append(0)

        argument.append([])

    for index in range(len(src_churn)):
        argument[index].append(src_churn[index])
        argument[index].append(team_size[index])
        argument[index].append(file_churn[index])
        argument[index].append(test_churn[index])
    
    return np.array(argument), np.array(build_result)

In [21]:
def output_values(Y_data):
    Y_t = []
    for e in Y_data:
        if e == 'passed':
            Y_t.append(1)
        else:
            Y_t.append(0) 
    return Y_t

In [22]:
def get_first_failures(df):
    
    results = df['tr_status'].tolist()
    length = len(results)
    verdict = ['keep']
    prev = results[0]
    
    for i in range(1, length):
        if results[i] == 0:
            if prev == 0:
                verdict.append('discard')
                #print(i+1)
            else:
                verdict.append('keep')
        else:
            verdict.append('keep')
        prev = results[i]
    
    df['verdict'] = verdict
    df = df[ df['verdict'] == 'keep' ]
    df.drop('verdict', inplace=True, axis=1)
    return df

In [23]:
def pd_get_train_test_data(file_path):
    columns = ['tr_build_id', 'gh_team_size', 'git_diff_src_churn', 'git_diff_test_churn', 'gh_diff_files_modified', 'tr_status']
    X = pd.read_csv(file_path, usecols = columns)
    X['tr_status'] = output_values(X['tr_status'])
    Y = X['tr_status']
    #X = get_first_failures(X)
    #X.drop('tr_status', inplace=True, axis=1)
    
    return X, Y

In [52]:
def sbs(project):
    
    #dataset already has first failures
    train_file = "../data/train_data/" + project + '_train.csv'
    num_feature = 4
    
    X_train, Y_train = pd_get_train_test_data(train_file)
    
    #X_train = X_train.reshape((int(len(X_train)), num_feature+1))
    print(X_train)
    print(Y_train)
    
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    max_depth = [int(x) for x in np.linspace(10, 110, num = 5)]
    
    rf = RandomForestClassifier()
    param_grid = {'n_estimators': n_estimators, 'max_depth': max_depth}
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 0)
    
    best_n_estimators = []
    best_max_depth = []
    
    best_f1 = 0
    best_f1_sample = 0
    best_f1_sample_result = 0
    best_f1_estimator = 0
    best_thresholds = []
    
    for i in range(1):
        print('Bootstrapping {} for {}'.format(i, project))
        
        while True:
            print('Here for {} {}'.format(i, project))
            sample_train = resample(X_train, replace=True, n_samples=len(X_train))
            sample_train_result = sample_train['tr_status']

            build_ids = sample_train['tr_build_id'].tolist()
            sample_test = X_train [~X_train['tr_build_id'].isin(build_ids)] 
            sample_test_result = sample_test['tr_status']

            if len(sample_test_result) != 0:
                break
        
        sample_train.drop('tr_status', inplace=True, axis=1)
        sample_train.drop('tr_build_id', inplace=True, axis=1)
        sample_test.drop('tr_status', inplace=True, axis=1)
        sample_test.drop('tr_build_id', inplace=True, axis=1)
        
        print('Training {} for {}'.format(i, project))
        grid_search.fit(sample_train, sample_train_result)
        sample_pred_vals = grid_search.predict_proba(sample_test)

        pred_vals = sample_pred_vals[:, 1]
        fpr, tpr, t = roc_curve(sample_test_result, pred_vals)
        gmeans = sqrt(tpr * (1-fpr))
        ix = argmax(gmeans)
        bt = t[ix]
        best_thresholds.append(bt)
        
        final_pred_result = []
        #threshold setting
        for j in range(len(pred_vals)):
            if pred_vals[j] > bt:
                final_pred_result.append(1)
            else:
                final_pred_result.append(0)
        
        try:
            f1 = f1_score(sample_test_result, final_pred_result)
        except:
            print('')

        if f1 > best_f1:
            best_f1 = f1
            best_f1_sample = sample_train
            best_f1_sample_result = sample_train_result
            best_f1_estimator = grid_search.best_estimator_
            print(best_f1_sample)
            
        best_n_estimators.append(grid_search.best_params_['n_estimators'])
        best_max_depth.append(grid_search.best_params_['max_depth'])
        
    #completed with bootstrapping 
    threshold = median(best_thresholds)
    n_estimator = median(best_n_estimators)
    max_depth = median(best_max_depth)

    #retrain to get the best model
    forest = RandomForestClassifier(n_estimators=int(n_estimator), max_depth=int(max_depth))
    forest.fit(best_f1_sample, best_f1_sample_result)
                
    #predictor = rf.fit(X_train, Y_train)
    return forest, threshold

In [53]:
def get_durations(project):
    csv_file = pd.read_csv(project)
    durations = csv_file['tr_duration'].tolist()
    return durations

In [54]:
batch_total = 0
batch_durations = 0

In [55]:
def batch_bisect(grouped_batch, actual_group_results, durations):
    global batch_total
    global batch_durations
    
    batch_total += 1
    batch_durations += max(durations)
    
    if len(grouped_batch) == 1:
        return
    
    if 0 in actual_group_results:
        half_batch = len(grouped_batch)//2
        batch_bisect(grouped_batch[:half_batch], actual_group_results[:half_batch], durations[:half_batch])
        batch_bisect(grouped_batch[half_batch:], actual_group_results[half_batch:], durations[half_batch:])

In [56]:
def batch_stop_4(grouped_batch, actual_group_results, durations):
    global batch_total
    global batch_durations
    
    batch_total += 1
    batch_durations += max(durations)
    
    if len(grouped_batch) <= 4:
        batch_total += 4
        batch_durations += sum(durations)
        return
    
    if 0 in actual_group_results:
        half_batch = len(grouped_batch)//2
        batch_stop_4(grouped_batch[:half_batch], actual_group_results[:half_batch], durations[:half_batch])
        batch_stop_4(grouped_batch[half_batch:], actual_group_results[half_batch:], durations[half_batch:])

In [57]:
def static_rule(p):
    
    global batch_total
    global batch_durations
    
    p = p.split('.')[0]

    predictor, threshold = sbs(p)

    #get the test data
    
    test_file = "../data/test_data/" + p + '_test.csv'
    X_test, Y_test = pd_get_train_test_data(test_file)
    Y_duration = get_durations(test_file)
    
    X_test.drop('tr_build_id', inplace=True, axis=1)
    X_test.drop('tr_status', inplace=True, axis=1)
    
    
    for alg in algorithm:
        for max_batch_size in MAX_BATCH:
                        
            if alg == 'BATCH4':
                if max_batch_size != 4:
                    continue
            
            if alg == 'BATCHSTOP4':
                if max_batch_size < 4:
                    continue
                    
            


            Y_result = []
            grouped_batch = []
            actual_group_results = []
            group_duration = []
            num_feature = 4 
            length_of_test = len(Y_test)

            project_reqd_builds = []
            project_missed_builds = []
            project_build_duration = []
            project_saved_builds = []
            project_delays = []
            project_bad_builds = []
            project_batch_delays = []

            print('Processing {}'.format(p))
            for c in confidence:
                
                batch_delays = 0

                pass_streak = Y_test[0]
                total_builds = Y_test[0]
                missed_builds = 0
                miss_indexes = []
                build_indexes = []
                delay_durations = []

                if pass_streak == 0:
                    total_duration = Y_duration[0]
                    saved_builds = 0
                else:
                    total_duration = 0
                    saved_builds = 1

                index = 1
                while index < len(X_test):
                    commit = X_test.iloc[index]
                    print(commit)
                    predict = predictor.predict_proba([commit])
                    #we're setting a confidence of 'c' builds on SBS, if more than 'c' passes have been suggested in a row, we don't want to trust sbs
                    
                    if predict[0][1] > threshold:
                        value = 1
                    else:
                        value = 0
                    
                    
                    if pass_streak < c :
                        
                        if value == 0:
                            
                            while True:

                                grouped_batch = list(X_test[index : index+max_batch_size])
                                actual_group_results = list(Y_test[index : index+max_batch_size])
                                group_duration = Y_duration[index : index+max_batch_size]

                #                 if len(grouped_batch) < max_batch_size:
                #                     grouped_batch.append(index)
                #                     actual_group_results.append(Y_test[index])
                #                     group_duration.append(Y_duration[index])
                                if alg == 'BATCH4':
                                    if len(grouped_batch) == max_batch_size:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        total_builds += 1
                                        total_duration += max(group_duration)

                                        if 0 in actual_group_results:
                                            total_builds += max_batch_size
                                            total_duration += sum(group_duration)

                                        grouped_batch.clear()
                                        actual_group_results.clear()
                                        group_duration.clear()

                                elif alg == 'BATCHBISECT':
                                    if len(grouped_batch) == max_batch_size:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_total = 0
                                        batch_durations = 0

                                        batch_bisect(grouped_batch, actual_group_results, group_duration)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        total_builds += batch_total
                                        total_duration += batch_durations

                                        grouped_batch.clear()
                                        actual_group_results.clear()
                                elif alg == 'BATCHSTOP4':
                                    if len(grouped_batch) == max_batch_size:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_total = 0
                                        batch_durations = 0

                                        batch_stop_4(grouped_batch, actual_group_results, group_duration)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        total_builds += batch_total
                                        total_duration += batch_durations

                                        grouped_batch.clear()
                                        actual_group_results.clear()
                                        group_duration.clear()
                                elif alg == 'NOBATCH':
                                    if len(grouped_batch) == max_batch_size:
                                        if len(miss_indexes) > 0:
                                            if miss_indexes[-1] < index:
                                                for l in range(len(miss_indexes)):
                                                    e = miss_indexes.pop()
                                                    delay_durations.append(index - e + 1)

                                        batch_delays += max_batch_size*(max_batch_size-1)/2
                                        total_builds += 1
                                        total_duration += max(group_duration)

                                        grouped_batch.clear()
                                        actual_group_results.clear()
                                        group_duration.clear()

                                index += max_batch_size
                                pass_streak = 1                            

                                if 0 in actual_group_results:
                                    index += max_batch_size
                                else:
                                    break
                                
#                             total_builds += 1
#                             total_duration += Y_duration[index]
#                             if len(miss_indexes) > 0:
#                                 if miss_indexes[-1] < index:
#                                     for l in range(len(miss_indexes)):
#                                         e = miss_indexes.pop()
#                                         delay_durations.append(index - e + 1)
                        else:
                            pass_streak += 1
                            saved_builds += 1
                            if Y_test[index] == 0:
                                missed_builds += 1
                                miss_indexes.append(index)

                            #seeing only one build
                            index += 1

                    else:
                        
                        while True:

                            grouped_batch = list(X_test[index : index+max_batch_size])
                            actual_group_results = list(Y_test[index : index+max_batch_size])
                            group_duration = Y_duration[index : index+max_batch_size]

            #                 if len(grouped_batch) < max_batch_size:
            #                     grouped_batch.append(index)
            #                     actual_group_results.append(Y_test[index])
            #                     group_duration.append(Y_duration[index])

                            if alg == 'BATCH4':
                                if len(grouped_batch) == max_batch_size:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)
                                    
                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    total_builds += 1
                                    total_duration += max(group_duration)

                                    if 0 in actual_group_results:
                                        total_builds += max_batch_size
                                        total_duration += sum(group_duration)

                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    group_duration.clear()

                            elif alg == 'BATCHBISECT':
                                if len(grouped_batch) == max_batch_size:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = 0
                                    batch_durations = 0

                                    batch_bisect(grouped_batch, actual_group_results, group_duration)

                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    total_builds += batch_total
                                    total_duration += batch_durations

                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    group_duration.clear()
                            elif alg == 'BATCHSTOP4':
                                if len(grouped_batch) == max_batch_size:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)

                                    batch_total = 0
                                    batch_durations = 0

                                    batch_stop_4(grouped_batch, actual_group_results, group_duration)

                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    total_builds += batch_total
                                    total_duration += batch_durations

                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    group_duration.clear()
                            elif alg == 'NOBATCH':
                                if len(grouped_batch) == max_batch_size:
                                    if len(miss_indexes) > 0:
                                        if miss_indexes[-1] < index:
                                            for l in range(len(miss_indexes)):
                                                e = miss_indexes.pop()
                                                delay_durations.append(index - e + 1)
                                    
                                    batch_delays += max_batch_size*(max_batch_size-1)/2
                                    total_builds += 1
                                    total_duration += max(group_duration)

                                    grouped_batch.clear()
                                    actual_group_results.clear()
                                    group_duration.clear()

                            index += max_batch_size
                            pass_streak = 1
                            
                            if 0 in actual_group_results:
                                index += max_batch_size
                            else:
                                break
                    

        #         print('\tFor confidence {}:'.format(c))
        #         print('\t\tTotal builds needed : {}'.format(total_builds))
        #         print('\t\tTotal number of missed builds : {}'.format(missed_builds))
        #         print('\t\tTotal number of saved builds : {}'.format(saved_builds))
        #         print('\t\tTotal duration of builds : {}'.format(total_duration))
        #         print('\t\tTotal delays: {}'.format(delay_durations))

                project_reqd_builds.append(total_builds)
                project_missed_builds.append(missed_builds)
                project_build_duration.append(total_duration)
                project_saved_builds.append(saved_builds)
                project_delays.append(delay_durations)
                project_batch_delays.append(batch_delays)

            print(p)
            print(project_reqd_builds)
            print(project_missed_builds)
            print(project_build_duration)
            print(project_saved_builds)
            print(project_delays)
            print(project_batch_delays)
            
            for i in range(len(confidence)):
                print([p, alg, max_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, project_build_duration[i], 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i]])
                writer.writerow([p, alg, max_batch_size, confidence[i], 100*project_reqd_builds[i]/length_of_test, 100*project_missed_builds[i]/length_of_test, project_build_duration[i], 100*project_saved_builds[i]/length_of_test, project_delays[i], length_of_test, project_batch_delays[i]])

In [58]:
jobs = []
for p in projects[1:2]:
    static_rule(p)

      tr_build_id  gh_team_size  git_diff_src_churn  git_diff_test_churn  \
0          869280            47               40445                10464   
1          870133            47                   0                    0   
2          872592            47                  22                    2   
3          872776            47                   0                    2   
4          872956            47                  23                    0   
...           ...           ...                 ...                  ...   
4035    156262711            35                   0                    0   
4036    156265417            35                   0                    0   
4037    156465458            35                2057                   89   
4038    156647755            35                   2                   13   
4039    156656143            35                  62                    0   

      gh_diff_files_modified  tr_status  
0                        593          0  
1  

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


      gh_team_size  git_diff_src_churn  git_diff_test_churn  \
1203            46                   2                    0   
3395            37                   0                    0   
1845            37                   3                    0   
2698            41                   0                   47   
3685            38                   5                    0   
...            ...                 ...                  ...   
3006            41                   1                    0   
13              48                 223                    0   
2161            37                  39                    0   
1606            37                   6                    0   
2177            38                 124                    0   

      gh_diff_files_modified  
1203                       1  
3395                       1  
1845                       1  
2698                       3  
3685                       2  
...                      ...  
3006                      

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: User

gh_team_size              41
git_diff_src_churn        22
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 15, dtype: int64
gh_team_size              41
git_diff_src_churn         0
git_diff_test_churn        7
gh_diff_files_modified     0
Name: 19, dtype: int64
gh_team_size              41
git_diff_src_churn         2
git_diff_test_churn        0
gh_diff_files_modified     2
Name: 20, dtype: int64
gh_team_size              41
git_diff_src_churn        23
git_diff_test_churn        0
gh_diff_files_modified     4
Name: 24, dtype: int64
gh_team_size              41
git_diff_src_churn        12
git_diff_test_churn       19
gh_diff_files_modified     2
Name: 25, dtype: int64


...

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: User

gh_team_size              42
git_diff_src_churn         5
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 145, dtype: int64
gh_team_size               42
git_diff_src_churn        313
git_diff_test_churn         0
gh_diff_files_modified     20
Name: 146, dtype: int64
gh_team_size              40
git_diff_src_churn         3
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 1, dtype: int64
gh_team_size              42
git_diff_src_churn         5
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 145, dtype: int64
gh_team_size               42
git_diff_src_churn        313
git_diff_test_churn         0
gh_diff_files_modified     20
Name: 146, dtype: int64


/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: User

gh_team_size              40
git_diff_src_churn         3
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 1, dtype: int64
gh_team_size              42
git_diff_src_churn         5
git_diff_test_churn        0
gh_diff_files_modified     1
Name: 145, dtype: int64
gh_team_size               42
git_diff_src_churn        313
git_diff_test_churn         0
gh_diff_files_modified     20
Name: 146, dtype: int64
jruby
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['jruby', 'BATCHSTOP4', 16, 2, 0.24213075060532688, 0.0, 0, 0.48426150121065376, [], 413, 0]
['jruby', 'BATCHSTOP4', 16, 3, 0.24213075060532688, 0.0, 0, 0.48426150121065376, [], 413, 0]
['jruby', 'BATCHSTO

/opt/homebrew/Caskroom/miniforge/base/envs/new_env/lib/python3.9/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [33]:
for q in jobs:
    q.terminate()

In [17]:
static_rule('loomio')

Processing loomio
loomio
[69, 53, 43, 36, 31, 27, 24, 23, 20, 19, 17, 17, 16, 14, 14, 13, 13, 13, 11]
[11, 11, 15, 20, 17, 16, 22, 20, 22, 23, 20, 22, 21, 22, 20, 24, 23, 22, 23]
[82428, 62599, 51430, 42929, 37185, 31615, 27426, 27745, 23396, 22318, 18535, 19911, 18917, 15262, 16215, 14960, 14374, 15036, 12449]
[69, 101, 121, 135, 145, 153, 159, 161, 167, 169, 173, 173, 175, 179, 179, 181, 181, 181, 185]
[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2], [3, 3, 2, 3, 4, 3, 2, 4, 2, 4, 3, 4, 2, 4, 3], [5, 5, 3, 5, 4, 3, 3, 2, 4, 2, 3, 5, 5, 2, 4, 3, 5, 5, 5, 4], [3, 5, 6, 2, 6, 2, 2, 5, 4, 4, 6, 2, 5, 3, 5, 5, 6], [5, 7, 2, 7, 5, 7, 3, 4, 5, 7, 2, 3, 5, 7, 3, 6], [2, 5, 7, 3, 2, 4, 6, 2, 7, 7, 6, 2, 6, 8, 2, 4, 2, 4, 6, 8, 8, 4], [3, 3, 5, 2, 3, 6, 3, 9, 2, 6, 4, 3, 7, 9, 4, 6, 5, 7, 9, 3], [4, 10, 10, 2, 10, 6, 2, 4, 10, 4, 8, 6, 4, 8, 10, 3, 6, 8, 8, 10, 7, 5], [5, 5, 7, 6, 7, 4, 9, 11, 3, 9, 4, 8, 6, 5, 9, 11, 3, 5, 8, 10, 11, 11, 10], [6, 11, 2, 5, 7, 6, 2, 6, 11

loomio
[23, 21, 19, 18, 16, 16, 14, 13, 13, 12, 11, 11, 10, 10, 10, 9, 9, 9, 9]
[4, 3, 7, 12, 6, 11, 14, 17, 9, 16, 15, 13, 15, 16, 14, 17, 16, 14, 19]
[29133, 27084, 23807, 22714, 19616, 19985, 17427, 15674, 15960, 14558, 13295, 13496, 11835, 11709, 12182, 10806, 11133, 11066, 10908]
[24, 41, 58, 69, 81, 85, 97, 105, 109, 117, 118, 121, 131, 133, 133, 141, 141, 141, 141]
[[2, 2, 2, 2], [3, 2, 3], [4, 4, 4, 4, 2, 3, 3], [3, 5, 3, 2, 2, 3, 5, 2, 4, 5, 5, 4], [5, 5, 3, 5, 3, 5], [3, 2, 7, 3, 6, 5, 3, 5, 7, 3, 4], [2, 2, 4, 6, 2, 3, 5, 6, 6, 3, 5, 7, 6], [3, 7, 9, 2, 7, 9, 3, 5, 2, 3, 7, 9, 3, 5, 7, 5], [4, 7, 2, 9, 2, 6, 8, 7, 9], [5, 6, 3, 5, 3, 9, 2, 10, 5, 9, 11, 5, 7, 9, 11, 11], [6, 3, 5, 11, 4, 9, 11, 3, 8, 10, 6, 3, 10, 11], [7, 7, 9, 5, 3, 10, 6, 10, 2, 4, 11, 13], [8, 11, 13, 2, 10, 3, 9, 5, 13, 2, 4, 11, 2, 14], [9, 4, 15, 5, 7, 15, 2, 7, 9, 15, 6, 4, 6, 13, 5], [10, 8, 5, 10, 12, 4, 8, 13, 15, 2, 5, 7, 14, 7], [11, 12, 3, 10, 15, 17, 2, 10, 14, 2, 4, 11, 15, 17, 5, 17], [12, 2

loomio
[97, 81, 63, 46, 47, 45, 30, 33, 26, 23, 27, 23, 24, 20, 24, 15, 17, 19, 15]
[11, 11, 15, 20, 17, 16, 22, 20, 22, 23, 20, 22, 21, 22, 20, 24, 23, 22, 23]
[114961, 95015, 74422, 54486, 55978, 52407, 34216, 40573, 30355, 27147, 28677, 26331, 28072, 22047, 27983, 17151, 19084, 22756, 16990]
[69, 101, 121, 135, 145, 153, 159, 161, 167, 169, 173, 173, 175, 179, 179, 181, 181, 181, 185]
[[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], [3, 2, 3, 3, 3, 3, 2, 3, 3, 3, 2], [3, 3, 2, 3, 4, 3, 2, 4, 2, 4, 3, 4, 2, 4, 3], [5, 5, 3, 5, 4, 3, 3, 2, 4, 2, 3, 5, 5, 2, 4, 3, 5, 5, 5, 4], [3, 5, 6, 2, 6, 2, 2, 5, 4, 4, 6, 2, 5, 3, 5, 5, 6], [5, 7, 2, 7, 5, 7, 3, 4, 5, 7, 2, 3, 5, 7, 3, 6], [2, 5, 7, 3, 2, 4, 6, 2, 7, 7, 6, 2, 6, 8, 2, 4, 2, 4, 6, 8, 8, 4], [3, 3, 5, 2, 3, 6, 3, 9, 2, 6, 4, 3, 7, 9, 4, 6, 5, 7, 9, 3], [4, 10, 10, 2, 10, 6, 2, 4, 10, 4, 8, 6, 4, 8, 10, 3, 6, 8, 8, 10, 7, 5], [5, 5, 7, 6, 7, 4, 9, 11, 3, 9, 4, 8, 6, 5, 9, 11, 3, 5, 8, 10, 11, 11, 10], [6, 11, 2, 5, 7, 6, 2, 6, 11, 6, 6, 10, 12, 3

loomio
[127, 129, 109, 82, 110, 90, 64, 55, 85, 58, 57, 65, 56, 50, 64, 43, 49, 59, 41]
[4, 3, 7, 12, 6, 11, 14, 17, 9, 16, 15, 13, 15, 16, 14, 17, 16, 14, 19]
[158856, 164022, 134219, 103709, 135232, 110525, 79759, 68834, 104943, 72913, 69259, 81913, 70899, 58773, 76863, 52349, 62410, 74336, 51150]
[24, 41, 58, 69, 81, 85, 97, 105, 109, 117, 118, 121, 131, 133, 133, 141, 141, 141, 141]
[[2, 2, 2, 2], [3, 2, 3], [4, 4, 4, 4, 2, 3, 3], [3, 5, 3, 2, 2, 3, 5, 2, 4, 5, 5, 4], [5, 5, 3, 5, 3, 5], [3, 2, 7, 3, 6, 5, 3, 5, 7, 3, 4], [2, 2, 4, 6, 2, 3, 5, 6, 6, 3, 5, 7, 6], [3, 7, 9, 2, 7, 9, 3, 5, 2, 3, 7, 9, 3, 5, 7, 5], [4, 7, 2, 9, 2, 6, 8, 7, 9], [5, 6, 3, 5, 3, 9, 2, 10, 5, 9, 11, 5, 7, 9, 11, 11], [6, 3, 5, 11, 4, 9, 11, 3, 8, 10, 6, 3, 10, 11], [7, 7, 9, 5, 3, 10, 6, 10, 2, 4, 11, 13], [8, 11, 13, 2, 10, 3, 9, 5, 13, 2, 4, 11, 2, 14], [9, 4, 15, 5, 7, 15, 2, 7, 9, 15, 6, 4, 6, 13, 5], [10, 8, 5, 10, 12, 4, 8, 13, 15, 2, 5, 7, 14, 7], [11, 12, 3, 10, 15, 17, 2, 10, 14, 2, 4, 11, 15, 17,

loomio
[153, 161, 139, 108, 136, 116, 74, 73, 103, 72, 71, 81, 60, 60, 80, 49, 59, 69, 59]
[4, 3, 7, 12, 6, 11, 14, 17, 9, 16, 15, 13, 15, 16, 14, 17, 16, 14, 19]
[190425, 200107, 169058, 135037, 163941, 140493, 91219, 91955, 124933, 91421, 87087, 100548, 74884, 70708, 95844, 59818, 73855, 86946, 74772]
[24, 41, 58, 69, 81, 85, 97, 105, 109, 117, 118, 121, 131, 133, 133, 141, 141, 141, 141]
[[2, 2, 2, 2], [3, 2, 3], [4, 4, 4, 4, 2, 3, 3], [3, 5, 3, 2, 2, 3, 5, 2, 4, 5, 5, 4], [5, 5, 3, 5, 3, 5], [3, 2, 7, 3, 6, 5, 3, 5, 7, 3, 4], [2, 2, 4, 6, 2, 3, 5, 6, 6, 3, 5, 7, 6], [3, 7, 9, 2, 7, 9, 3, 5, 2, 3, 7, 9, 3, 5, 7, 5], [4, 7, 2, 9, 2, 6, 8, 7, 9], [5, 6, 3, 5, 3, 9, 2, 10, 5, 9, 11, 5, 7, 9, 11, 11], [6, 3, 5, 11, 4, 9, 11, 3, 8, 10, 6, 3, 10, 11], [7, 7, 9, 5, 3, 10, 6, 10, 2, 4, 11, 13], [8, 11, 13, 2, 10, 3, 9, 5, 13, 2, 4, 11, 2, 14], [9, 4, 15, 5, 7, 15, 2, 7, 9, 15, 6, 4, 6, 13, 5], [10, 8, 5, 10, 12, 4, 8, 13, 15, 2, 5, 7, 14, 7], [11, 12, 3, 10, 15, 17, 2, 10, 14, 2, 4, 11, 15,

In [ ]:
[544, 548, 540, 547, 551, 551, 551, 539, 542, 546, 538, 545, 549, 549, 549, 537, 540, 544, 536]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[477479, 482294, 472759, 483396, 486160, 487122, 490128, 473004, 475976, 480767, 471197, 481910, 484623, 485591, 488642, 471611, 474392, 479182, 469789]
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]